# DeepSeek R1 Distillation 1.5B vLLM serving using the Azure ML Python SDK

> [1] Please use `Python 3.10 - SDK v2 (azureml_py310_sdkv2)` conda environment.<br>[2] Please make sure you prepare [Hugging Face API Token](https://huggingface.co/docs/hub/security-tokens).

## 1. Load config file

---


In [ ]:
# !huggingface-cli login --token <your token>

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /home/azureuser/.cache/huggingface/token
Login successful


In [24]:
!huggingface-cli download unsloth/DeepSeek-R1-GGUF --quiet --include "*UD-Q2_K_XL*" --local-dir DeepSeek-R1-GGUF --cache-dir .cache

/mnt/batch/tasks/shared/LS_root/mounts/clusters/jihualiu3/code/llm-inferencing/DeepSeek-R1-GGUF


In [ ]:
import os
import yaml
from logger import logger
from datetime import datetime

snapshot_date = datetime.now().strftime("%Y-%m-%d")

with open("config.yml") as f:
    d = yaml.load(f, Loader=yaml.FullLoader)

AZURE_SUBSCRIPTION_ID = d["config"]["AZURE_SUBSCRIPTION_ID"]
AZURE_RESOURCE_GROUP = d["config"]["AZURE_RESOURCE_GROUP"]
AZURE_WORKSPACE = d["config"]["AZURE_WORKSPACE"]
HF_TOKEN = ''
HF_MODEL_NAME_OR_PATH = d["config"]["HF_MODEL_NAME_OR_PATH"]
IS_DEBUG = d["config"]["IS_DEBUG"]

azure_env_name = 'deepseek-llm-cpp-r1-k'
azure_model_name = 'deepseek-r1-k'
azure_endpoint_name = 'deepseek-r1-k-endpoint'
azure_deployment_name = 'blue'
azure_serving_cluster_size = 'Standard_ND40rs_v2'


if IS_DEBUG:
    logger.debug("===== 0. Azure ML Deployment Info =====")
    logger.debug(f"AZURE_SUBSCRIPTION_ID={AZURE_SUBSCRIPTION_ID}")
    logger.debug(f"AZURE_RESOURCE_GROUP={AZURE_RESOURCE_GROUP}")
    logger.debug(f"AZURE_WORKSPACE={AZURE_WORKSPACE}")
    logger.debug(f"HF_MODEL_NAME_OR_PATH={HF_MODEL_NAME_OR_PATH}")
    logger.debug(f"IS_DEBUG={IS_DEBUG}")

    logger.debug(f"azure_env_name={azure_env_name}")
    logger.debug(f"azure_model_name={azure_model_name}")
    logger.debug(f"azure_endpoint_name={azure_endpoint_name}")
    logger.debug(f"azure_deployment_name={azure_deployment_name}")
    logger.debug(f"azure_serving_cluster_size={azure_serving_cluster_size}")

2025-03-05 13:51:39,450 - logger - DEBUG - ===== 0. Azure ML Deployment Info =====
2025-03-05 13:51:39,450 - logger - DEBUG - AZURE_SUBSCRIPTION_ID=<your-subscription-id>
2025-03-05 13:51:39,450 - logger - DEBUG - AZURE_RESOURCE_GROUP=<your-resource-group>
2025-03-05 13:51:39,450 - logger - DEBUG - AZURE_WORKSPACE=<your-workspace-name>
2025-03-05 13:51:39,450 - logger - DEBUG - HF_MODEL_NAME_OR_PATH=DeepSeek-R1-GGUF/DeepSeek-R1-UD-Q2_K_XL
2025-03-05 13:51:39,465 - logger - DEBUG - IS_DEBUG=True
2025-03-05 13:51:39,466 - logger - DEBUG - azure_env_name=deepseek-llm-cpp-r1-k
2025-03-05 13:51:39,466 - logger - DEBUG - azure_model_name=deepseek-r1-k
2025-03-05 13:51:39,466 - logger - DEBUG - azure_endpoint_name=deepseek-r1-k-endpoint
2025-03-05 13:51:39,466 - logger - DEBUG - azure_deployment_name=blue
2025-03-05 13:51:39,471 - logger - DEBUG - azure_serving_cluster_size=Standard_ND40rs_v2


<br>

## 2. Serving preparation

---

### 2.1. Configure workspace details

To connect to a workspace, we need identifying parameters - a subscription, a resource group, and a workspace name. We will use these details in the MLClient from azure.ai.ml to get a handle on the Azure Machine Learning workspace we need. We will use the default Azure authentication for this hands-on.


In [5]:
%pip install -q azure-ai-ml azure-identity

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
azureml-mlflow 1.57.0.post1 requires azure-storage-blob<=12.19.0,>=12.5.0, but you have azure-storage-blob 12.24.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [28]:
# import required libraries
import time
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient, Input
from azure.ai.ml import command
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.core.exceptions import ResourceNotFoundError, ResourceExistsError

logger.info(f"===== 2. Serving preparation =====")
logger.info(f"Calling DefaultAzureCredential.")
credential = DefaultAzureCredential()
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    ml_client = MLClient(
        credential, AZURE_SUBSCRIPTION_ID, AZURE_RESOURCE_GROUP, AZURE_WORKSPACE
    )

2025-03-02 14:47:12,281 - logger - INFO - ===== 2. Serving preparation =====
2025-03-02 14:47:12,749 - logger - INFO - Calling DefaultAzureCredential.
Found the config file in: /config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


### 2.2. Create model asset


In [31]:
def get_or_create_model_asset(
    ml_client,
    model_name,
    job_name=None,
    model_dir="outputs",
    model_type="custom_model",
    update=False,
):
    try:
        latest_model_version = max(
            [int(m.version) for m in ml_client.models.list(name=model_name)]
        )
        if update:
            raise ResourceExistsError("Found Model asset, but will update the Model.")
        else:
            model_asset = ml_client.models.get(
                name=model_name, version=latest_model_version
            )
            logger.info(f"Found Model asset: {model_name}. Will not create again")
    except (ResourceNotFoundError, ResourceExistsError, ValueError) as e:
        logger.info(f"Exception: {e}")
        if job_name is None:
            model_path = model_dir
        else:
            model_path = (
                f"azureml://jobs/{job_name}/outputs/artifacts/paths/{model_dir}/"
            )
        run_model = Model(
            name=model_name,
            path=model_path,
            description="Model created from run.",
            type=model_type,  # mlflow_model, custom_model, triton_model
        )
        model_asset = ml_client.models.create_or_update(run_model)
        logger.info(f"Created Model asset: {model_name}")

    return model_asset

In [ ]:
model = get_or_create_model_asset(
    ml_client,
    azure_model_name,
    job_name=None,
    model_dir="DeepSeek-R1-GGUF/DeepSeek-R1-UD-Q2_K_XL",
    model_type="custom_model",
    update=False,
)

2025-03-02 14:48:46,542 - logger - INFO - Exception: max() arg is an empty sequence


### 2.3. Create AzureML environment

Azure ML defines containers (called environment asset) in which your code will run. We can use the built-in environment or build a custom environment (Docker container, conda). This hands-on uses Docker container.


#### Docker environment


In [11]:
from azure.ai.ml.entities import Environment, BuildContext


def get_or_create_docker_environment_asset(
    ml_client, env_name, docker_dir, inference_config=None, update=False
):

    try:
        latest_env_version = max(
            [int(e.version) for e in ml_client.environments.list(name=env_name)]
        )
        if update:
            raise ResourceExistsError(
                "Found Environment asset, but will update the Environment."
            )
        else:
            env_asset = ml_client.environments.get(
                name=env_name, version=latest_env_version
            )
            logger.info(f"Found Environment asset: {env_name}. Will not create again")
    except (ResourceNotFoundError, ResourceExistsError) as e:
        logger.info(f"Exception: {e}")
        env_docker_image = Environment(
            build=BuildContext(path=docker_dir),
            name=env_name,
            description="Environment created from a Docker context.",
            inference_config=inference_config,
        )
        env_asset = ml_client.environments.create_or_update(env_docker_image)
        logger.info(f"Created Environment asset: {env_name}")

    return env_asset


inference_config = {
    "liveness_route": {
        "port": 8000,
        "path": "/health",
    },
    "readiness_route": {
        "port": 8000,
        "path": "/health",
    },
    "scoring_route": {
        "port": 8000,
        "path": "/",
    },
}

env = get_or_create_docker_environment_asset(
    ml_client, azure_env_name, "docker-r1", inference_config, update=True
)

2025-03-02 11:09:34,210 - logger - INFO - Exception: Found Environment asset, but will update the Environment.
2025-03-02 11:09:49,595 - logger - INFO - Created Environment asset: deepseek-llm-cpp-r1-k


<br>

## 3. Serving

---

### 3.1. Create endpoint

Create an endpoint. This process does not provision a GPU cluster yet.


In [ ]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
)

logger.info(f"===== 3. Serving =====")

t0 = time.time()

# Check if the endpoint already exists in the workspace
try:
    endpoint = ml_client.online_endpoints.get(azure_endpoint_name)
    logger.info("---Endpoint already exists---")
except:
    # Create an online endpoint if it doesn't exist

    # Define the endpoint
    endpoint = ManagedOnlineEndpoint(
        name=azure_endpoint_name,
        description=f"Test endpoint for {model.name}",
    )

# Trigger the endpoint creation
try:
    ml_client.begin_create_or_update(endpoint).wait()
    logger.info("\n---Endpoint created successfully---\n")
except Exception as err:
    raise RuntimeError(f"Endpoint creation failed. Detailed Response:\n{err}") from err

t1 = time.time()

from humanfriendly import format_timespan

timespan = format_timespan(t1 - t0)
logger.info(f"Creating Endpoint took {timespan}")

2025-03-03 00:44:57,284 - logger - INFO - ===== 3. Serving =====
2025-03-03 00:46:29,877 - logger - INFO - 
---Endpoint created successfully---

2025-03-03 00:46:29,878 - logger - INFO - Creating Endpoint took 1 minute and 32.59 seconds


### 3.2. Create Deployment

Create a Deployment. This takes a lot of time as GPU clusters must be provisioned and the serving environment must be built.


In [54]:
env_vars = {
    "MODEL_NAME": '/models/DeepSeek-R1-UD-Q2_K_XL/DeepSeek-R1-UD-Q2_K_XL-00001-of-00005.gguf', ## /var/azureml-app/azureml-models/deepseek-adapter/DeepSeek-R1-Distill-Qwen-1.5B
    "LAYER_N": "50",
    "PREDICT_N": "10000",
    "VLLM_ARGS": ""
}
deployment_env_vars = {**env_vars}

In [ ]:
%%time
import time
from azure.ai.ml.entities import (    
    OnlineRequestSettings,
    CodeConfiguration,
    ManagedOnlineDeployment,
    ProbeSettings,
    Environment
)

t0 = time.time()
deployment = ManagedOnlineDeployment(
    name=azure_deployment_name,
    endpoint_name=azure_endpoint_name,
    model=model,
    model_mount_path='/models',
    instance_type=azure_serving_cluster_size,
    instance_count=1,
    environment_variables=deployment_env_vars,    
    environment=env,
    request_settings=OnlineRequestSettings(
        max_concurrent_requests_per_instance=2,
        request_timeout_ms=180000, 
        max_queue_wait_ms=360000
    ),
    liveness_probe=ProbeSettings(
        failure_threshold=5,
        success_threshold=1,
        timeout=10,
        period=30,
        initial_delay=120
    ),
    readiness_probe=ProbeSettings(
        failure_threshold=30,
        success_threshold=1,
        timeout=2,
        period=10,
        initial_delay=120,
    ),
)

# Trigger the deployment creation
try:
    ml_client.begin_create_or_update(deployment).wait()
    logger.info("\n---Deployment created successfully---\n")
except Exception as err:
    raise RuntimeError(
        f"Deployment creation failed. Detailed Response:\n{err}"
    ) from err
    
endpoint.traffic = {azure_deployment_name: 100}
endpoint_poller = ml_client.online_endpoints.begin_create_or_update(endpoint)

t1 = time.time()
timespan = format_timespan(t1 - t0)
logger.info(f"Creating deployment took {timespan}")

Check: endpoint deepseek-r1-k-endpoint exists


..........................................................................................................................................................................................................................

2025-03-03 01:07:16,008 - logger - INFO - 
---Deployment created successfully---

2025-03-03 01:07:17,500 - logger - INFO - Creating deployment took 18 minutes and 53.22 seconds


CPU times: user 2.78 s, sys: 183 ms, total: 2.97 s
Wall time: 18min 53s


In [60]:
endpoint_results = endpoint_poller.result()

In [61]:
print(endpoint_results)

auth_mode: key
description: Test endpoint for deepseek-r1-k
id: /subscriptions/e56790f8-0506-49eb-95b8-82817828d59d/resourceGroups/dev/providers/Microsoft.MachineLearningServices/workspaces/test-vllm/onlineEndpoints/deepseek-r1-k-endpoint
identity:
  principal_id: fd1d162f-8998-49bc-a173-1fd86eac0546
  tenant_id: 16b3c013-d300-468d-ac64-7eda0820b6d3
  type: system_assigned
kind: Managed
location: westeurope
mirror_traffic: {}
name: deepseek-r1-k-endpoint
openapi_uri: https://deepseek-r1-k-endpoint.westeurope.inference.ml.azure.com/swagger.json
properties:
  AzureAsyncOperationUri: https://management.azure.com/subscriptions/e56790f8-0506-49eb-95b8-82817828d59d/providers/Microsoft.MachineLearningServices/locations/westeurope/mfeOperationsStatus/oeidp:10331bed-c16d-4680-834e-e054c4eeaeb9:70d1ddb2-a0e4-47b7-89b4-637b17ad918c?api-version=2022-02-01-preview
  azureml.onlineendpointid: /subscriptions/e56790f8-0506-49eb-95b8-82817828d59d/resourcegroups/dev/providers/microsoft.machinelearningse

In [62]:
endpoint_name = endpoint_results.name
keys = ml_client.online_endpoints.get_keys(name=endpoint_name)
primary_key = keys.primary_key

<br>

## 4. Test

---

### 4.1. Invocation

Try calling the endpoint.


In [40]:
%pip install -q openai

Note: you may need to restart the kernel to use updated packages.


In [63]:
from openai import OpenAI

url = os.path.join(endpoint_results.scoring_uri, "v1")
endpoint_name = (
    endpoint_results.name if azure_endpoint_name is None else azure_endpoint_name
)
keys = ml_client.online_endpoints.get_keys(name=endpoint_name)
primary_key = keys.primary_key  # You can paste [YOUR Azure ML API KEY] here
llm = OpenAI(base_url=url, api_key=primary_key)
model_path = "/models/DeepSeek-R1-GGUF/DeepSeek-R1-UD-Q2_K_XL"

In [64]:
# Create your prompt
system_message = """
You are an AI assistant that helps customers find information. As an assistant, you respond to questions in a concise and unique manner.
You can use Markdown to answer simply and concisely, and add a personal touch with appropriate emojis.

Add a witty joke starting with "By the way," at the end of your response. Do not mention the customer's name in the joke part.
The joke should be related to the specific question asked.
For example, if the question is about tents, the joke should be specifically related to tents.

Use the given context to provide a more personalized response. Write each sentence on a new line:
"""
context = """
    The Alpine Explorer Tent features a detachable partition to ensure privacy, 
    numerous mesh windows and adjustable vents for ventilation, and a waterproof design. 
    It also includes a built-in gear loft for storing outdoor essentials. 
    In short, it offers a harmonious blend of privacy, comfort, and convenience, making it a second home in nature!
"""
question = "What are features of the Alpine Explorer Tent?"

user_message = f"""
Context: {context}
Question: {question}
"""

Simple API Call

In [65]:
# Simple API Call
response = llm.chat.completions.create(
    model=model_path,
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0.7,
    max_tokens=200,
)

print(response.choices[0].message.content)

<think>
Okay, let's tackle this question. The user is asking about the features of the Alpine Explorer Tent. First, I need to go through the provided context to pick out the key points. The context mentions a detachable partition for privacy, mesh windows, adjustable vents for ventilation, a waterproof design, and a built-in gear loft. Also, it says it's like a second home in nature.

Now, I should present these features clearly and concisely. Each feature on a new line as per the user's instruction. Then, add a joke related to camping or tents. The joke needs to start with "By the way," and not mention the customer's name. Let me think of a tent-related pun. Maybe something about campers and poles, like "why don’t campers argue? They prefer to stick to their poles!" That works because tents have poles and it's a play on words. I'll place the joke at the end after the emoji


Streaming

In [70]:
response = llm.chat.completions.create(
    model=model_path,
    messages=[
        {"role": "saystem", "content": system_message},
        {"role": "user", "content": user_message},
    ],
    temperature=0.7,
    max_tokens=6000,
    stream=True,  # Stream the response
)

print("Streaming response:")
for chunk in response:
    delta = chunk.choices[0].delta
    if hasattr(delta, "content"):
        print(delta.content, end="", flush=True)

Streaming response:
<think>
Okay, let me see. I need to figure out the features of the Alpine Explorer Tent based on the context provided. Let me read through the context again carefully.

The context says it has a detachable partition for privacy. That's one feature. Then there are numerous mesh windows and adjustable vents for ventilation. So ventilation is covered by those. Also, it's a waterproof design, which is important for a tent. There's a built-in gear loft for storing outdoor essentials. The conclusion mentions a blend of privacy, comfort, and convenience, making it like a second home. 

So the main points are: detachable partition, mesh windows, adjustable vents, waterproof design, built-in gear loft. I should list these as features. The user might want each feature clearly listed, maybe in bullet points. Let me make sure I didn't miss anything. The context mentions "in short" but the features before that are the key points. The gear loft is specific for storage. Privacy th

Another method

In [66]:
import requests

completions_url = os.path.join(endpoint_results.scoring_uri, "v1/completions")
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {primary_key}"}
data = {
    "model": model_path,
    "prompt": "San Francisco is a ",
    "max_tokens": 200,
    "temperature": 0.7,
}

response = requests.post(completions_url, headers=headers, json=data, timeout=300)
print(response.json())

{'choices': [{'text': '7×7-mile city located at the tip of a peninsula between the Pacific Ocean and San Francisco Bay. The city is known for its steep hills, eclectic mix of architecture, and landmarks, including the Golden Gate Bridge, cable cars, Alcatraz Island, and its Chinatown district.\n\nThe Golden Gate Bridge is a suspension bridge spanning the Golden Gate, the one-mile-wide strait connecting San Francisco Bay and the Pacific Ocean. The bridge is one of the most internationally recognized symbols of the United States and is one of the most photographed bridges in the world.\n\nAlcatraz Island is located in San Francisco Bay, 1.5 miles offshore from San Francisco. The small island was the site of a military prison from 1859 to 1933, and a federal prison from 1934 to 1963. Today, the island is a national recreation area and is open to visitors.\n\nChinatown in San Francisco is the oldest and largest Chinatown in North America. The', 'index': 0, 'logprobs': None, 'finish_reason'

### 4.2. LLM latency/throughput simple benchmarking


In [67]:
import numpy as np
from time import perf_counter


def simple_llm_benchmark(
    llm: OpenAI,
    messages: list,
    model_path: str = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    num_warmups: int = 1,
    num_infers: int = 5,
    **params: dict,
) -> dict:

    print("=== Measuring latency ===")
    print(f"model_path={model_path}, num_infers={num_infers}, params={params}")

    latencies = []
    # Warm up
    for _ in range(num_warmups):
        response = llm.chat.completions.create(
            model=model_path,
            messages=messages,
            **params,
        )
    print("=== Warmup done. Start Benchmarking... ===")
    begin = time.time()
    # Timed run
    for curr_infer in range(num_infers):
        start_time = perf_counter()
        if (curr_infer % 5) == 0:
            print(f"Inferring {curr_infer}th...")
        response = llm.chat.completions.create(
            model=model_path,
            messages=messages,
            **params,
        )
        latency = perf_counter() - start_time
        latencies.append(latency)
    end = time.time()

    # Compute run statistics
    duration = end - begin
    time_avg_sec = np.mean(latencies)
    time_std_sec = np.std(latencies)
    time_p95_sec = np.percentile(latencies, 95)
    time_p99_sec = np.percentile(latencies, 99)

    # Metrics
    metrics = {
        "duration": duration,
        "avg_sec": time_avg_sec,
        "std_sec": time_std_sec,
        "p95_sec": time_p95_sec,
        "p99_sec": time_p99_sec,
    }

    return metrics

In [68]:
messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message},
]
params = {
    "max_tokens": 100,
    "temperature": 0.5,
}

model_path = "/models/DeepSeek-R1-GGUF/DeepSeek-R1-UD-Q2_K_XL"

metrics = simple_llm_benchmark(
    llm, messages, model_path=model_path, num_warmups=1, num_infers=10, **params
)

=== Measuring latency ===
model_path=/models/deepseek-adapter/DeepSeek-R1-Distill-Qwen-1.5B, num_infers=10, params={'max_tokens': 100, 'temperature': 0.5}
=== Warmup done. Start Benchmarking... ===
Inferring 0th...
Inferring 5th...


In [69]:
import pprint

pprint.pprint(metrics)

{'avg_sec': 24.593713035504333,
 'duration': 245.93714904785156,
 'p95_sec': 27.716650118958203,
 'p99_sec': 28.64820212058723,
 'std_sec': 2.31673521689894}


## Clean up


In [71]:
ml_client.online_endpoints.begin_delete(azure_endpoint_name)

...................................................................................